# Atmospheric retrieval with petitRADTRANS

This is a tutorial for atmospheric retrieval with [petitRADTRANS](https://petitradtrans.readthedocs.io/) for which we will use NIR spectra and photometry of the planetary-mass companion [ROXs 42 Bb](https://ui.adsabs.harvard.edu/abs/2014ApJ...780L..30C/abstract). Free retrievals are computationally expensive due to the high number of parameter dimensions and the scattering radiative transfer that is important for cloudy objects (see [Mollière et al. 2020](https://ui.adsabs.harvard.edu/abs/2020A%26A...640A.131M/abstract)). Similar [FitModel](https://species.readthedocs.io/en/latest/species.analysis.html#species.analysis.fit_model.FitModel), the nested sampling with the free retrievals uses multiprocessing so it recommended to run such retrievals on a cluster. Before starting, ``petitRADTRANS`` should be manually installed together with the line and continuum opacities.

When running a retrieval, it is important to comment out any of the functions that access the [Database](https://species.readthedocs.io/en/latest/species.data.html#species.data.database.Database) because writing to the HDF5 database is not possible with multiprocessing. Therefore, the companion data should first be added to the database, then [AtmosphericRetrieval](https://species.readthedocs.io/en/latest/species.analysis.html#species.analysis.retrieval.AtmosphericRetrieval) and [run_multinest](https://species.readthedocs.io/en/latest/species.analysis.html#species.analysis.retrieval.AtmosphericRetrieval.run_multinest) can be executed (e.g. on a cluster), and finally the results can be extracted and plotted while commenting out the retrieval part.

## Getting started

We start by importing [urllib](https://docs.python.org/3/library/urllib.html) and 
the `species` toolkit.

In [1]:
import urllib
import species

Let's now download the GRAVITY $K$ band spectrum of beta Pic b that was published by [Gravity Collaboration et al. 2020](https://ui.adsabs.harvard.edu/abs/2020A%26A...633A.110G/abstract) and we will also make use of the GPI $YJH$ band spectra from [Chilcote et al. 2017](https://ui.adsabs.harvard.edu/abs/2017AJ....153..182C/abstract).

In [3]:
urllib.request.urlretrieve('https://home.strw.leidenuniv.nl/~stolker/species/BetaPictorisb_2018-09-22.fits',
                           'BetaPictorisb_2018-09-22.fits')
urllib.request.urlretrieve('https://home.strw.leidenuniv.nl/~stolker/species/betapicb_gpi_y.dat',
                           'betapicb_gpi_y.dat')
urllib.request.urlretrieve('https://home.strw.leidenuniv.nl/~stolker/species/betapicb_gpi_j.dat',
                           'betapicb_gpi_j.dat')
urllib.request.urlretrieve('https://home.strw.leidenuniv.nl/~stolker/species/betapicb_gpi_h.dat',
                           'betapicb_gpi_h.dat')

('betapicb_gpi_h.dat', <http.client.HTTPMessage at 0x14d8aab50>)

Next, we need to add the library path of ``MultiNest`` to the ``DYLD_LIBRARY_PATH`` environment variable such that ``PyMultiNest`` can find the compiled library (see [installation instructions](https://johannesbuchner.github.io/PyMultiNest/install.html)).

In [4]:
import os
os.environ['DYLD_LIBRARY_PATH'] = '/Users/tomasstolker/applications/MultiNest/lib'

We are now ready to use the `species` toolkit by creating an instance of [SpeciesInit](https://species.readthedocs.io/en/latest/species.core.html#species.core.init.SpeciesInit) class. This will create the HDF5 database and [configuration file](https://species.readthedocs.io/en/latest/configuration.html) in the working. If these files were already present, then the existing database and configuration will be used.

In [17]:
species.SpeciesInit()

Initiating species v0.4.0... [DONE]
Database: /Users/tomasstolker/applications/species/docs/tutorials/species_database.hdf5
Data folder: /Users/tomasstolker/applications/species/docs/tutorials/data
Working folder: /Users/tomasstolker/applications/species/docs/tutorials


## Adding observational data to the database

The database is used by ``species`` as the central storage for models, data, and results. The data in the HDF5 file is accessed through the [Database](https://species.readthedocs.io/en/latest/species.data.html#species.data.database.Database) class. It is best to add the data beforehand and comment out the database part when executing the code on a cluster with multi-core processing.

In [18]:
database = species.Database()

We will now add spectra and distance of [beta Pic b](https://exoplanets.nasa.gov/exoplanet-catalog/7040/beta-pictoris-b/) to the database. This is done with the [add_object](https://species.readthedocs.io/en/latest/species.data.html#species.data.database.add_object) method by providing a dictionary as argument of `spectrum`. The tuple of each spectrum name contains the filename with the spectrum, optionally the covariance matrix, and the spectral resolution (that is used for smoothing the model spectra).

In [19]:
database.add_object(object_name='beta Pic b',
                    distance=(19.75, 0.13),
                    app_mag=None,
                    spectrum={'GPI-Y': ('betapicb_gpi_y.dat', None, 35.),
                              'GPI-J': ('betapicb_gpi_j.dat', None, 37.),
                              'GPI-H': ('betapicb_gpi_h.dat', None, 47.),
                              'GRAVITY': ('BetaPictorisb_2018-09-22.fits', 'BetaPictorisb_2018-09-22.fits', 500.)})

Adding object: beta Pic b
   - Distance (pc) = 19.75 +/- 0.13
   - Spectrum:
      - Database tag: GPI-Y
      - Filename: betapicb_gpi_y.dat
      - Data shape: (29, 3)
      - Wavelength range (um): 0.98 - 1.13
      - Mean flux (W m-2 um-1): 5.40e-15
      - Mean error (W m-2 um-1): 1.87e-15
   - Spectrum:
      - Database tag: GPI-J
      - Filename: betapicb_gpi_j.dat
      - Data shape: (32, 3)
      - Wavelength range (um): 1.13 - 1.34
      - Mean flux (W m-2 um-1): 6.84e-15
      - Mean error (W m-2 um-1): 4.79e-16
   - Spectrum:
      - Database tag: GPI-H
      - Filename: betapicb_gpi_h.dat
      - Data shape: (34, 3)
      - Wavelength range (um): 1.51 - 1.79
      - Mean flux (W m-2 um-1): 5.63e-15
      - Mean error (W m-2 um-1): 3.86e-16
   - GRAVITY spectrum:
      - Object: Unknown
      - Database tag: GRAVITY
      - Filename: BetaPictorisb_2018-09-22.fits
      - Data shape: (237, 3)
      - Wavelength range (um): 1.97 - 2.49
      - Mean flux (W m-2 um-1): 4.65e-1

In this tutorial, we will not make use of the photometric data. These are We will now add the photometric data but these could also be included with the retrieval. There is a collection of magnitudes and fluxes [available in species](https://github.com/tomasstolker/species/blob/master/species/data/companions.py). These can be automatically added to the database with the [add_companion](https://species.readthedocs.io/en/latest/species.data.html#species.data.database.Database.add_companion) method, for example `add_companion('beta Pic b')`.

## Atmospheric retrieval of abundances, clouds, and P-T structure

We are now ready to start the retrieval! We start by creating an instance of [AtmosphericRetrieval](https://species.readthedocs.io/en/latest/species.data.html#species.analysis.retrieval.AtmosphericRetrieval). Here we provide among others the database tag of the planet data, the line and cloud species that should be included in the forward model, and if scattering should be turned on with the radiative transfer. Scattering will make the calculation of the forward model much slower but is important for cloudy directly imaged objects.

In [15]:
retrieve = species.AtmosphericRetrieval(object_name='beta Pic b',
                                        line_species=['CO_all_iso_HITEMP', 'H2O_HITEMP', 'CH4', 'NH3', 'CO2', 'H2S', 'Na_allard', 'K_allard', 'PH3', 'VO_Plez', 'TiO_all_Exomol', 'FeH'],
                                        cloud_species=['MgSiO3(c)_cd', 'Fe(c)_cd'],
                                        scattering=True,
                                        output_folder='multinest',
                                        wavel_range=(1.1, 2.46),
                                        inc_spec=['GPI-Y', 'GPI-J', 'GPI-H', 'GRAVITY'],
                                        inc_phot=False,
                                        pressure_grid='smaller',
                                        weights=None)

Object: beta Pic b
Distance: 19.75
Line species:
   - CO_all_iso_HITEMP
   - H2O_HITEMP
   - CH4
   - NH3
   - CO2
   - H2S
   - Na_allard
   - K_allard
   - PH3
   - VO_Plez
   - TiO_all_Exomol
   - FeH
Cloud species:
   - MgSiO3(c)_cd
   - Fe(c)_cd
Line-by-line species: None
Scattering: True
Getting object: beta Pic b... [DONE]
Spectroscopic data:
   - GPI-H
     Wavelength range (um) = 1.51 - 1.79
     Spectral resolution = 47.00
   - GPI-J
     Wavelength range (um) = 1.13 - 1.34
     Spectral resolution = 37.00
   - GPI-Y
     Wavelength range (um) = 0.98 - 1.13
     Spectral resolution = 35.00
   - GRAVITY
     Wavelength range (um) = 1.97 - 2.49
     Spectral resolution = 500.00
Initiating 180 pressure levels (bar): 1.00e-06 - 1.00e+03
Weights for the log-likelihood function:
   - GPI-Y = 1.00e+00
   - GPI-J = 1.00e+00
   - GPI-H = 1.00e+00
   - GRAVITY = 1.00e+00


In [ ]:
retrieve.run_multinest(bounds={'logg': (2., 6.),
                               'c_o_ratio': (0.1, 1.5),
                               'metallicity': (-3., 3.),
                               'radius': (0.1, 5.),
                               'fsed': (0., 10.),
                               'log_kzz': (1., 15.),
                               'sigma_lnorm': (1.05, 5.),
                               'mgsio3_fraction': (-3., 1.),
                               'fe_fraction': (-3., 1.),
                               'GPI-Y': ((0.5, 1.5), None, None),
                               'GPI-J': ((0.5, 1.5), None, None),
                               'GPI-H': ((0.5, 1.5), None, None)},
                       chemistry='equilibrium',
                       quenching='pressure',
                       pt_profile='molliere',
                       n_live_points=1000,
                       resume=True,
                       plotting=False,
                       pt_smooth=None,
                       check_flux=None,
                       temp_nodes=None,
                       prior={'mass': (9., 1.6)})

## Nested sampling output folder

The output data from the nested sampling with ``MultiNest`` is stored in the ``output_folder``. We can use the [add_retrieval](https://species.readthedocs.io/en/latest/species.data.html#species.data.database.Database.add_retrieval) method of [Database](https://species.readthedocs.io/en/latest/species.data.html#species.data.database.Database) to store the posterior samples and relevant attributes in the HDF5 database. The argument of ``tag`` is used as name tag in the database. It is also possible to calculate $T_\mathrm{eff}$ for each sample but this takes a long time because each spectrum needs to be calculated over a broad wavelength range.

In [ ]:
database.add_retrieval(tag='roxs42bb',
                       output_folder='multinest',
                       inc_teff=False)

Instead, we use the [get_retrieval_teff](https://species.readthedocs.io/en/latest/species.data.html#species.data.database.Database.add_retrieval) to estimate $T_\mathrm{eff}$ from a small number of samples. The value is stored in the database as attribute of the ``tag`` group.

In [ ]:
database.get_retrieval_teff(tag='roxs42bb',
                            random=30)

## Plotting the posterior distributions

We can now read the posterior samples from the database and use the plot functionalities to visualize the results. Let's first plot the marginalized posterior distributions by using the [corner.py](https://corner.readthedocs.io) package. The plot is created with the [plot_posterior](https://species.readthedocs.io/en/latest/species.plot.html#species.plot.plot_mcmc.plot_posterior) function. Since there are many free parameters, we will leave out those for the P-T profile by setting `inc_pt_param=False`.

In [ ]:
species.plot_posterior(tag='roxs42bb',
                       offset=(-0.3, -0.35),
                       vmr=False,
                       inc_mass=False,
                       inc_pt_param=False,
                       output='posterior.png')

Let's have a look at the corner plot!

In [ ]:
from IPython.display import Image
Image('posterior.png')

## ReadRadtrans and random spectra

In order to post-process the posterior samples, we need to recreate the ``Radtrans`` object of ``petitRADTRANS``. We will use the [get_retrieval_spectra](https://species.readthedocs.io/en/latest/species.data.html#species.data.database.Database.get_retrieval_spectra) method of [Database](https://species.readthedocs.io/en/latest/species.data.html#species.data.database.Database) to create and instance of [ReadRadtrans](https://species.readthedocs.io/en/latest/species.read.html#species.read.read_radtrans.ReadRadtrans) with the adopted parameters from the retrieval. The ``Radtrans`` object is stored as an attribute of ``ReadRadtrans`` and will be used by ``species`` but can typically be ignored by the user. The method also returns a list of random spectra (30 in the example below) that have been recalculated at a resolving power of $R = 2000$. Each of the model spectra is stored in a [ModelBox](https://species.readthedocs.io/en/latest/species.core.html#species.core.box.ModelBox).

In [ ]:
samples, radtrans = database.get_retrieval_spectra(tag='roxs42bb',
                                                   random=30,
                                                   wavel_range=(0.5, 6.),
                                                   spec_res=2000.)

# Create plots of P-T profiles, opacities, and clouds

In [ ]:
species.plot_pt_profile(tag='roxs42bb',
                        random=100,
                        xlim=(0., 6000.),
                        offset=(-0.07, -0.14),
                        output='plot/pt_profile_grains.pdf',
                        radtrans=radtrans,
                        extra_axis='grains')

species.plot_opacities(tag='roxs42bb',
                       offset=(-0.1, -0.14),
                       output='plot/opacities.pdf',
                       radtrans=radtrans)

species.plot_clouds(tag='roxs42bb',
                    offset=(-0.1, -0.15),
                    output='plot/clouds.pdf',
                    radtrans=radtrans,
                    composition='MgSiO3')

## Read companion data, best-fit sample, and fit residuals

In [ ]:
best = database.get_probable_sample(tag='roxs42bb')

objectbox = database.get_object('ROXs 42 Bb',
                                inc_phot=False)

objectbox = species.update_spectra(objectbox, best)

residuals = species.get_residuals(datatype='model',
                                  spectrum='petitradtrans',
                                  parameters=best,
                                  objectbox=objectbox,
                                  inc_phot=False,
                                  inc_spec=True,
                                  radtrans=radtrans)

modelbox = radtrans.get_model(model_param=best,
                              spec_res=2000.,
                              plot_contribution='plot/contribution.pdf')

no_clouds = best.copy()
no_clouds['log_tau_cloud'] = -100.
model_no_clouds = radtrans.get_model(no_clouds)

## Plot the SED with data and models

In [ ]:
species.plot_spectrum(boxes=[samples, modelbox, model_no_clouds, objectbox],
                      filters=None,
                      plot_kwargs=[{'ls': '-', 'lw': 0.1, 'color': 'gray'},
                                   {'ls': '-', 'lw': 0.5, 'color': 'black'},
                                   {'ls': '--', 'lw': 0.3, 'color': 'black'},
                                   {'NIFS': {'marker': 'o', 'ms': 2., 'color': 'tab:green', 'ls': 'none', 'alpha': 0.2, 'mew': 0., 'label': 'NIFS'},
                                    'OSIRIS': {'marker': 'o', 'ms': 2., 'color': 'tab:blue', 'ls': 'none', 'alpha': 0.2, 'mew': 0., 'label': 'OSIRIS'},
                                    'SINFONI': {'marker': 'o', 'ms': 2., 'color': 'tab:orange', 'ls': 'none', 'alpha': 0.2, 'mew': 0., 'label': 'SINFONI'}}],
                      residuals=residuals,
                      xlim=(1.1, 2.5),
                      ylim=(0.15e-16, 1.15e-15),
                      ylim_res=(-5., 5.),
                      scale=('linear', 'linear'),
                      offset=(-0.6, -0.05),
                      figsize=(8, 2.5),
                      legend=[{'loc': 'upper right', 'fontsize': 8.}, {'loc': 'lower left', 'fontsize': 8.}],
                      output='plot/spectrum.pdf')

## Plot the SED over broader wavelength range

In [ ]:
modelbox = radtrans.get_model(model_param=best,
                              spec_res=200.,
                              plot_contribution='plot/contribution.pdf')

objectbox = database.get_object('ROXs 42 Bb',
                                inc_phot=True)

objectbox = species.update_spectra(objectbox, best)

for item in ['NIFS', 'OSIRIS', 'SINFONI']:
    objectbox.spectrum[item] = (objectbox.spectrum[item][0][::50, ],
                                objectbox.spectrum[item][1],
                                objectbox.spectrum[item][2],
                                objectbox.spectrum[item][3])

residuals = species.get_residuals(datatype='model',
                                  spectrum='petitradtrans',
                                  parameters=best,
                                  objectbox=objectbox,
                                  inc_phot=True,
                                  inc_spec=True,
                                  radtrans=radtrans)

synphot = species.multi_photometry(datatype='model',
                                   spectrum='petitradtrans',
                                   filters=objectbox.filters,
                                   parameters=best,
                                   radtrans=radtrans)

object_kwargs = {'NIFS': {'marker': 'o', 'ms': 2.5, 'color': 'tab:green', 'ls': 'none', 'alpha': 0.5, 'mew': 0., 'label': 'Gemini/NIFS'},
                 'OSIRIS': {'marker': 'o', 'ms': 2.5, 'color': 'tab:blue', 'ls': 'none', 'alpha': 0.5, 'mew': 0., 'label': 'Keck/OSIRIS'},
                 'SINFONI': {'marker': 'o', 'ms': 2.5, 'color': 'tab:orange', 'ls': 'none', 'alpha': 0.5, 'mew': 0., 'label': 'VLT/SINFONI'}}

for item in objectbox.magnitude:
    object_kwargs[item] = {'marker': 's', 'ms': 3.5, 'color': 'black'}

species.plot_spectrum(boxes=[samples, modelbox, objectbox, synphot],
                      filters=None,
                      plot_kwargs=[{'ls': '-', 'lw': 0.1, 'color': 'gray'},
                                   {'ls': '-', 'lw': 0.5, 'color': 'black'},
                                   object_kwargs,
                                   None],
                      residuals=residuals,
                      xlim=(0.8, 5.),
                      ylim=(0.15e-15, 1.9e-15),
                      ylim_res=(-7., 7.),
                      scale=('linear', 'linear'),
                      offset=(-0.6, -0.04),
                      figsize=(8, 2.5),
                      legend=[None, {'loc': 'upper right', 'fontsize': 10.}],
                      quantity='flux',
                      output='plot/full_sed.pdf')